# Step 1
Import all necessary modules
Fill in the paths, species designations, and AOI name below

In [8]:
import pandas as pd
import laspy
import geopandas as gpd
import json
import os
import shutil
import subprocess
import numpy as np
import plotly.express as px
from pptx import Presentation
from pptx.util import Inches
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.util import Pt
from python_pptx_text_replacer import TextReplacer
from pptx.enum.text import PP_ALIGN
import datetime

####### WARNING, if you are running Windows 10, you need to downgrade kaleido module like this :
# pip install --upgrade "kaleido==0.1.*"
agtLAS = r"C:\--directory--\AOI_merged.las"
agtSHP = r"C:\--direcotry\AOI_mergedInitialShapes.shp"
outputDirectory = r"C:\--directory--\IntensityGraphs"
#decid = ['AW','PB','BW']
decid = ['AH','AR','BE','BI','CH','FM','HA','HC','MA','OK','PO','SY','WL']
#conifer = ['FB','LT','PL','SB','SW']
conifer = ['MC']
dead = ['DP','SN']
aoiName = 'AOI'

# Step 2
Access Intensity Data

In [9]:
# Locate LASFromPoly tool
if os.path.exists(os.environ['LOCALAPPDATA'] + '\\Dropbox\\info.json'):
    with open(os.environ['LOCALAPPDATA'] + '\\Dropbox\\info.json') as f:
        LASFromPoly_Path = json.load(f)['business']['path'] + r"\Project Data\OR\_TSI30Release\LASFromPoly.exe"
elif os.path.exists(os.environ['APPDATA'] + '\\Dropbox\\info.json'):
    with open(os.environ['APPDATA'] + '\\Dropbox\\info.json') as f:
        LASFromPoly_Path = json.load(f)['business']['path'] + r"\Project Data\OR\_TSI30Release\LASFromPoly.exe"
else:
    print("Dropbox\\info.json location not not found")
    exit()

# Reclassify LAZ into LAS using laszip
if agtLAS[-1] == 'z':
    agtDir = os.path.dirname(agtLAS) # Get directory of agtLAS
    subprocess.run("laszip -i *.laz",cwd=agtDir) # Convert LAZ to LAS
    old_agtLAS = agtLAS.replace(".laz",".las")
    baseName = os.path.basename(old_agtLAS)
    agtLAS = os.path.join(outputDirectory,baseName)
    shutil.copy(old_agtLAS,agtLAS)


las = laspy.read(agtLAS)
shp = gpd.read_file(agtSHP)
las.header.add_crs(shp.crs)
shp['SPECIES'] = shp['NAME'].str[-2:]
shp_unique = shp['SPECIES'].unique()

shp_count = list() # Create list for species count

for i in shp_unique:
    shp_temp = shp[shp['SPECIES'] == i]
    shp_count.append(shp_temp.shape[0])
    temp_int = list()
    temp_class = list()
    os.makedirs(f"{outputDirectory}\individualLAS",exist_ok=True)
    os.makedirs(f"{outputDirectory}\individualLAS\{i}",exist_ok=True)
    shp_temp.to_file(f"{outputDirectory}\individualLAS\{i}.shp")
    
    # Prepare LASFromPoly command
    args1 = [ 
        LASFromPoly_Path,
        os.path.dirname(agtLAS),
        f"{outputDirectory}\individualLAS\{i}.shp",
        fr"{outputDirectory}\individualLAS\{i}",
        "nThreads=4"
    ]
    #subprocess.run(args1, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    subprocess.run(args1)
    
    # Create CSV files with intensity lists
    speciesLAS_list = os.listdir(f"{outputDirectory}\individualLAS\{i}")
    speciesLAS_list = [os.path.join(f"{outputDirectory}\individualLAS\{i}",f) for f in speciesLAS_list]
    
    # Create data frame with intensity and classifications
    for c_l in speciesLAS_list:
        temp_las = laspy.read(c_l)
        [temp_int.append(intensity) for intensity in temp_las.intensity.tolist()]
        [temp_class.append(classification) for classification in list(temp_las.classification)]
    temp_df = pd.DataFrame({"Intensity":temp_int,"Classification":temp_class})
    temp_df = temp_df[temp_df['Classification']!=2]
    #temp_df = temp_df[temp_df['Classification']==1] # Use when the LAS only has classifications of 1 and 2 (2 is ground)
    os.makedirs(f"{outputDirectory}\Raw_Intensity",exist_ok=True)
    temp_df.to_csv(f"{outputDirectory}\Raw_Intensity\{i}_rawIntensity.csv")
    
# Create species count CSV
spec_sum = pd.DataFrame({'Species': shp_unique, 'Count': shp_count})
spec_sum.to_csv(f"{outputDirectory}\speciesCount.csv",index=False)

# Step 3
Create Frequency Tables

In [10]:
# Create frequency bins
intensity_max = las.intensity.max()
if intensity_max < 5000: # Use the optimal frequency bin size for the data
    by_n = 200
elif 5000 <= intensity_max <= 25000:
    by_n = 1000
elif 25000 < intensity_max <= 75000:
    by_n = 2000
else:
    by_n = 5000
br = list(range(0, intensity_max + by_n, by_n))

# Create intensity frequency table
csv_files = [f for f in os.listdir(os.path.join(outputDirectory, "Raw_Intensity")) if f.endswith("rawIntensity.csv")] # Access list of rawIntensity files
totalIntensity = pd.DataFrame()
for i in csv_files:
    temp_data = pd.read_csv(os.path.join(outputDirectory, "Raw_Intensity", i))
    temp_name = i.replace("_rawIntensity.csv", "").upper()
    freq, edges = np.histogram(temp_data['Intensity'], bins=br) # Frequency histogram

    if 'Bin' not in totalIntensity.columns:
        totalIntensity['Bin'] = edges[:-1]
    totalIntensity[f'Frequency_{temp_name}'] = freq # Separate frequencies into bins
    totalIntensity[f'Percentage_{temp_name}'] = (totalIntensity[f'Frequency_{temp_name}'] / totalIntensity[f'Frequency_{temp_name}'].sum()) * 100 # Add percentage column
    totalIntensity[f'species_{temp_name}'] = temp_name

totalIntensity.to_csv(f"{outputDirectory}\IntensityHistogram.csv")

# Step 4
Prepare Intensity Plots and save them

In [11]:
df_percent = totalIntensity.loc[:, totalIntensity.columns.str.startswith('Percentage')]
df_percent['Bin'] = range(0,len(df_percent))

# Prepare dataset with decimal percentages
df_final = pd.DataFrame()
for col in df_percent.columns[:-1]:
    temp_name = col[-2:].upper()
    temp_df = pd.DataFrame({'Percentage': df_percent[col], 'SSP': temp_name, 'Intensity': df_percent['Bin']})
    temp_df = temp_df.drop(0)
    df_final = pd.concat([df_final, temp_df], ignore_index=True)
df_final['Percentage'] = df_final['Percentage'] * 0.01

##### Display data
col_pal = px.colors.qualitative.Light24 # Create colour palette
ssp_unique = df_final['SSP'].unique() # Get list of unique species in the order they appear

# All Species plot
all_col_pal = col_pal[0:len(ssp_unique)]
fig_all = px.line(df_final,x='Intensity',y='Percentage',color='SSP', labels={'color': "SSP"}, color_discrete_sequence=all_col_pal)
fig_all.update_layout(yaxis=dict(tickformat="%"), title={'text': f"{aoiName} - All Species Intensity Histogram",
        'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'}, legend_title="Species",
    title_x=0.5, font=dict(size=15), plot_bgcolor = "white",yaxis_tickformat=".0%", title_font_color='black')
fig_all.update_xaxes(gridcolor="white",showline=True, linewidth=2, linecolor='black')
fig_all.update_yaxes(gridcolor="black",showline=True, linewidth=2, linecolor='black')
fig_all.show()

# Deciduous plot
dc_df = df_final[df_final['SSP'].isin(decid)]
dc_idx = [x for x,z in enumerate(np.char.upper(ssp_unique.astype(str))) if z in decid] # get index of DC species
dc_col_pal = [all_col_pal[i] for i in dc_idx] # configure colour palette for DC species
fig_dc = px.line(dc_df,x='Intensity',y='Percentage',color='SSP', labels={'color': "SSP"}, color_discrete_sequence=dc_col_pal)
fig_dc.update_layout(yaxis=dict(tickformat="%"), title={'text': f"{aoiName} - Deciduous Species Intensity Histogram",
        'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'}, legend_title="Species",
    title_x=0.5, font=dict(size=15), plot_bgcolor = "white",yaxis_tickformat=".0%", title_font_color='black')
fig_dc.update_xaxes(gridcolor="white",showline=True, linewidth=2, linecolor='black')
fig_dc.update_yaxes(gridcolor="black",showline=True, linewidth=2, linecolor='black')
fig_dc.show()

# Coniferous plot
cn_df = df_final[df_final['SSP'].isin(conifer)]
cn_idx = [x for x,z in enumerate(np.char.upper(ssp_unique.astype(str))) if z in conifer] # get index of CN species
cn_col_pal = [all_col_pal[i] for i in cn_idx] # configure colour palette for CN species
fig_cn = px.line(cn_df,x='Intensity',y='Percentage',color='SSP', labels={'color': "SSP"}, color_discrete_sequence=cn_col_pal)
fig_cn.update_layout(yaxis=dict(tickformat="%"), title={'text': f"{aoiName} - Coniferous Species Intensity Histogram",
        'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'}, legend_title="Species",
    title_x=0.5, font=dict(size=15), plot_bgcolor = "white",yaxis_tickformat=".0%", title_font_color='black')
fig_cn.update_xaxes(gridcolor="white",showline=True, linewidth=2, linecolor='black')
fig_cn.update_yaxes(gridcolor="black",showline=True, linewidth=2, linecolor='black')
fig_cn.show()

# Dead plot
de_df = df_final[df_final['SSP'].isin(dead)]
de_idx = [x for x,z in enumerate(np.char.upper(ssp_unique.astype(str))) if z in dead] # get index of DE species
de_col_pal = [all_col_pal[i] for i in de_idx] # configure colour palette for DE species
fig_de = px.line(de_df,x='Intensity',y='Percentage',color='SSP', labels={'color': "SSP"}, color_discrete_sequence=de_col_pal)
fig_de.update_layout(yaxis=dict(tickformat="%"), title={'text': f"{aoiName} - Dead Species Intensity Histogram",
        'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'}, legend_title="Species",
    title_x=0.5, font=dict(size=15), plot_bgcolor = "white",yaxis_tickformat=".0%", title_font_color='black')
fig_de.update_xaxes(gridcolor="white",showline=True, linewidth=2, linecolor='black')
fig_de.update_yaxes(gridcolor="black",showline=True, linewidth=2, linecolor='black')
fig_de.show()

# Save plots
os.makedirs(f"{outputDirectory}\plots",exist_ok=True)
fig_all.write_image(f"{outputDirectory}\plots\{aoiName}_allSpeciesPlot.png",scale=3)
fig_dc.write_image(f"{outputDirectory}\plots\{aoiName}_decidSpeciesPlot.png",scale=3)
fig_cn.write_image(f"{outputDirectory}\plots\{aoiName}_coniferSpeciesPlot.png",scale=3)
fig_de.write_image(f"{outputDirectory}\plots\{aoiName}_deadSpeciesPlot.png",scale=3)

C:\Users\mburnett\AppData\Local\Temp\ipykernel_1108\149196335.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Step 5
Generate Powerpoint

In [12]:
# Locate Intensity Graphs Template
if os.path.exists(os.environ['LOCALAPPDATA'] + '\\Dropbox\\info.json'):
    with open(os.environ['LOCALAPPDATA'] + '\\Dropbox\\info.json') as f:
        template = json.load(f)['business']['path'] + r"\--directory--\TEMPLATE_IntensityGraphs.pptx"
elif os.path.exists(os.environ['APPDATA'] + '\\Dropbox\\info.json'):
    with open(os.environ['APPDATA'] + '\\Dropbox\\info.json') as f:
        template = json.load(f)['business']['path'] + r"\--directory--\TEMPLATE_IntensityGraphs.pptx"

### Prepare first slide
# Get today's date
currentDate = datetime.date.today()
currentMonth = currentDate.strftime("%B")
currentDay = currentDate.strftime("%d")
currentYear = currentDate.strftime("%Y")

# Replace text in the title slide
replacer = TextReplacer(template, slides='',
                        tables=True, charts=True, textframes=True)
replacer.replace_text( [ ('AOI Intensity Graphs ',f"{aoiName} Intensity Graphs ") ] )
replacer.replace_text( [ ('DATE',f"{currentMonth} {currentDay}, {currentYear}") ] )
replacer.write_presentation_to_file(f"{outputDirectory}\{aoiName}_IntensityGraphs.pptx")


#### Prepare the rest of the slides
presentation = Presentation(f"{outputDirectory}\{aoiName}_IntensityGraphs.pptx")
# Set image dimensions
top =Inches(0.1)
left = Inches(0.5)
width = Inches(10.5)
height = Inches(6.8)

# Set slide 1
all_species_slide = presentation.slides[1]
all_species_img = all_species_slide.shapes.add_picture(f"{outputDirectory}\plots\{aoiName}_allSpeciesPlot.png",left,top,width,height)
species_table = all_species_slide.shapes.add_table(len(spec_sum)+1,2,Inches(10.9),Inches(0.5),Inches(1),Inches(1)).table
species_table.columns[0].width = Inches(1.2)
species_table.columns[1].width = Inches(1.2)

spec_sum = spec_sum.sort_values('Species')
# Populate the table with data
for i in range(0,2):
    print(i)
    cell_header = species_table.cell(0,i)
    cell_header.text = str(spec_sum.columns.values[i])
for i in range(0,len(spec_sum)):
    cell0 = species_table.cell(i+1,0)
    cell1 = species_table.cell(i+1,1)
    cell0.alignment = PP_ALIGN.LEFT
    cell1.alignment = PP_ALIGN.LEFT
    #cell0.fill.solid()
    #cell0.fill.back_color = (255,255,255) # white background colour
    #cell1.fill.solid()
    #cell1.fill.back_colour = (255,255,255)
    cell0.text = str(spec_sum.iloc[i,0]).upper()
    cell1.text = str(spec_sum.iloc[i,1])
# Change table style
#tbl =  species_table._element.graphic.graphicData.tbl
#style_id = '{93296810-A885-4BE3-A3E7-6D5BEEA58F35}'
#tbl[0][-1].text = style_id

# Set slides 2-4
dc_species_slide = presentation.slides[2]
dc_species_img = dc_species_slide.shapes.add_picture(f"{outputDirectory}\plots\{aoiName}_decidSpeciesPlot.png",left,top,width,height)
cn_species_slide = presentation.slides[3]
cn_species_img = cn_species_slide.shapes.add_picture(f"{outputDirectory}\plots\{aoiName}_coniferSpeciesPlot.png",left,top,width,height)
de_species_slide = presentation.slides[4]
de_species_img = de_species_slide.shapes.add_picture(f"{outputDirectory}\plots\{aoiName}_deadSpeciesPlot.png",left,top,width,height)

# Save presentation
presentation.save(f"{outputDirectory}\{aoiName}_IntensityGraphs.pptx")



Slide[1].TEXT_BOX[id=10].Run[0,0]: 'AOI Intensity Graphs ' -> 'Fugro NGED Woodhall Intensity Graphs '
Slide[1].TEXT_BOX[id=12].Run[0,0]: 'DATE' -> 'January 19, 2024'
0
1
